In [1]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext()

In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [11]:
FILE_PATH = "dac/small-train.txt"
lines = sc.textFile(FILE_PATH)

In [12]:
lines.take(1)

[u'0\t1\t1\t5\t0\t1382\t4\t15\t2\t181\t1\t2\t\t2\t68fd1e64\t80e26c9b\tfb936136\t7b4723c4\t25c83c98\t7e0ccccf\tde7995b8\t1f89b562\ta73ee510\ta8cd5504\tb2cb9c98\t37c9c164\t2824a5f6\t1adce6ef\t8ba8b39a\t891b62e7\te5ba7672\tf54016b9\t21ddcdc9\tb1252a9d\t07b5194c\t\t3a171ecb\tc5c50484\te8b83407\t9727dd16']

In [13]:
data = lines.map(lambda line: line.split("\t"))

In [14]:
data.take(1)

[[u'0',
  u'1',
  u'1',
  u'5',
  u'0',
  u'1382',
  u'4',
  u'15',
  u'2',
  u'181',
  u'1',
  u'2',
  u'',
  u'2',
  u'68fd1e64',
  u'80e26c9b',
  u'fb936136',
  u'7b4723c4',
  u'25c83c98',
  u'7e0ccccf',
  u'de7995b8',
  u'1f89b562',
  u'a73ee510',
  u'a8cd5504',
  u'b2cb9c98',
  u'37c9c164',
  u'2824a5f6',
  u'1adce6ef',
  u'8ba8b39a',
  u'891b62e7',
  u'e5ba7672',
  u'f54016b9',
  u'21ddcdc9',
  u'b1252a9d',
  u'07b5194c',
  u'',
  u'3a171ecb',
  u'c5c50484',
  u'e8b83407',
  u'9727dd16']]